In [5]:
!pip install transformers==4.27.2
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.1 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [7]:
## Summarize the dialogue without the prompt engineering
## In this we will be generating a summary of a dialogue with the pre-trained
## LLM FLAN-T5 from hugging face
dataset = load_dataset("knkarthick/dialogsum")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
# Printing couple of dialogues with their baseline summaries

In [9]:
example_indices = [40,200]

dashline = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
  print(dashline)
  print('Example ',i+1)
  print(dashline)
  print("Input Dialogue : ")
  print(dataset['test'][index]['dialogue'])
  print(dashline)
  print("Bashline human summary : ")
  print(dataset['test'][index]['summary'])
  print(dashline)
  print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Dialogue : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Bashline human summary : 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------

In [10]:
## Load the flan t5 model, creating the instance of the automodelforseq2seqlm class with .from_pretrained() method

In [12]:
model_name =  'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [13]:
## To perform encoding and decoding we have to work with text in a tokenized form.
## Tokenization is the process of splitting texts into smaller units that can be processed by the LLM models

In [14]:
## Download the tokenizer for the flan t5 llm using the autotokenizer.from_pretrained() method.
## Parameter use_fast switches on fast tokenizer.

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

In [16]:
## Testing the tokenizer encoding and decoding a simple sentence

In [17]:
sentence = 'What time is it, Hank?'

sentence_encoded = tokenizer(sentence, return_tensors = 'pt')

sentence_decoded = tokenizer.decode(
    sentence_encoded['input_ids'][0],
    skip_special_tokens = True
)

print("ENCODED SENTENCE: ")
print(sentence_encoded['input_ids'][0])
print('\nDECODED SENTENCE: ')
print(sentence_decoded)

ENCODED SENTENCE: 
tensor([ 363,   97,   19,   34,    6, 6627,  157,   58,    1])

DECODED SENTENCE: 
What time is it, Hank?


In [18]:
# Now its time to explore how well llm summarizes a dialogue without any prompt engineering
## Prompt engineering is an act of a human changing the prompt or input to improve the response for a given task.

In [27]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  inputs = tokenizer(dialogue, return_tensors = 'pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens = 50,
      )[0],

      skip_special_tokens = True
  )

  print(dashline)
  print("Example ", i+1)
  print(dashline)
  print(f'Input Prompt: \n{dialogue}')
  print(dashline)
  print(f'Baseline Human Summary: \n{summary}')
  print(dashline)
  print(f'Model generation - Prompt Engineering: \n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Prompt: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model generation - Prompt Engineering: 
Person1: It's ten to nine.

------------------------------------

In [28]:
## We can see that model is not working upto to the mark, but with the help of prompt engineering we can improve the model

In [29]:
## First we will implement zero shot inference, in this we will take the dialogue and convert it into the instruction prompt

In [31]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']


  prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    ##Input constructed promppt instead of the dialogue
  inputs = tokenizer(prompt, return_tensors = 'pt')
  output = tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens = 50,
      )[0],
      skip_special_tokens = True
  )

  print(dashline)
  print("Example ", i+1)
  print(dashline)
  print(f'Input Prompt: \n{dialogue}')
  print(dashline)
  print(f'Baseline Human Summary: \n{summary}')
  print(dashline)
  print(f'Model generation - Prompt Engineering: \n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Prompt: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model generation - Prompt Engineering: 
The train is about to leave.

----------------------------------

In [33]:
## Model generation is still not upto the mark

In [34]:
## Now we will experiment with the prompt text and check how the inferences are changing/

In [35]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']


  prompt = f"""
Dialogie is : .

{dialogue}

Summary:
    """

    ##Input constructed promppt instead of the dialogue
  inputs = tokenizer(prompt, return_tensors = 'pt')
  output = tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens = 50,
      )[0],
      skip_special_tokens = True
  )

  print(dashline)
  print("Example ", i+1)
  print(dashline)
  print(f'Input Prompt: \n{dialogue}')
  print(dashline)
  print(f'Baseline Human Summary: \n{summary}')
  print(dashline)
  print(f'Model generation - Prompt Engineering: \n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Prompt: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model generation - Prompt Engineering: 
The train is about to leave.

----------------------------------

In [42]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']


  prompt = f"""
Dialogue

{dialogue}

What was going on?
"""

  ##Input constructed promppt instead of the dialogue
  inputs = tokenizer(prompt, return_tensors = 'pt')
  output = tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens = 50,
      )[0],
      skip_special_tokens = True
  )

  print(dashline)
  print("Example ", i+1)
  print(dashline)
  print(f'Input Prompt: \n{dialogue}')
  print(dashline)
  print(f'Baseline Human Summary: \n{summary}\n')
  print(dashline)
  print(f'Model generation - ZERO SHOT: \n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Prompt: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

---------------------------------------------------------------------------------------------------
Model generation - ZERO SHOT: 
Tom is late. He has to catch the nine-thirty train.

-------------------

In [43]:
## Now we use one shot and few shot inferences, these are the practices of providing an LLM with either one or more full examples
## of prompt-response pairs that matches our task before our actual prompt that we want completed.

In [45]:
def make_prompt(example_indices_full, example_index_to_summarize):
  prompt = ""
  for index in example_indices_full:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    ## The stop sequence '{summary}\n\n\n' is important for flan t5. other models may have their own preferred stop sequence.

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}

"""
  dialogue = dataset['test'][example_index_to_summarize]['dialogue']

  prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
  return prompt

In [46]:
# Construct the prompt to perform one shot inference

In [49]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt =  make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also nee

In [50]:
## Now pass this prompt to perform the one shot inference:

In [52]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens = True
)

print(dashline)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dashline)
print(f'MODEL GENERATION -- ONE SHOT:\n{output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION -- ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


In [53]:
## Now we dive into few shot inferences

In [54]:
example_indices_full = [40,80,120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. M

In [55]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens = True
)

print(dashline)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dashline)
print(f'MODEL GENERATION -- FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION -- FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.
